# Simulation

In [1]:
# Imports and utility

# Jupyter magic for autoreload:
%reload_ext autoreload
%autoreload 2

# Test sun trajectory calculation at runtime, NB. may take long!

import numpy as np
import matplotlib.pyplot as plt
import simpy

import lusee

# import nav
from nav        import * # includes luseepy
from bms.parts  import Battery, Controller
# ----------------------------------------------------
print(f'''simpy version: {simpy.__version__}''')
print(f'''lusee version: {lusee.__version__}''')


simpy version: 4.0.2
lusee version: 1.2-dev


In [11]:
# Read the prefab data on sun trajectory
import os
from nav import Sun
print('Current directory:', os.getcwd())

mySun = Sun()

cachefile = '../data/2025-02-04_03-07.npy'
mySun.read(cachefile)
print(f'''Number of points read from the file {cachefile}: {mySun.N}''')

sun = mySun.xyz # altaz2xyz(mySun.alt, mySun.az)
hrs = mySun.hrsFromSunrise()


Current directory: /home/maxim/projects/lusee/opsim/notebooks
Loaded data from file "../data/2025-02-04_03-07.npy", number of points for the three components: 8979
Number of points read from the file ../data/2025-02-04_03-07.npy: 2993


In [12]:
env = simpy.Environment()

battery = Battery() # a dummy battery here, just to create the controller

ctr     = Controller(battery, env)
ctr.add_all_panels(sun)
ctr.set_condition(mySun.condition)
ctr.set_time(-10)

env.process(ctr.run())

env.run(until=15)



Clock 0
Clock 1
Clock 2
Clock 3
Clock 4
Clock 5
Clock 6
Clock 7
Clock 8
Clock 9
Clock 10
Clock 11
Clock 12
Clock 13
Clock 14
